In [ ]:
CREATE STORAGE INTEGRATION CO2_S3_INTEGRATION
TYPE = EXTERNAL_STAGE
STORAGE_PROVIDER = 'S3'
STORAGE_AWS_ROLE_ARN = 'arn:aws:iam::183295451617:role/MySnowflakeRole'
ENABLED = TRUE
STORAGE_ALLOWED_LOCATIONS = ('s3://big.data.ass3/');


In [ ]:
SHOW INTEGRATIONS;
DESC INTEGRATION CO2_S3_INTEGRATION;


In [ ]:
CREATE OR REPLACE STAGE CO2_STAGE
STORAGE_INTEGRATION = CO2_S3_INTEGRATION
URL = 's3://big.data.ass3/';


In [ ]:
DESC STAGE CO2_STAGE;


In [ ]:
DESC INTEGRATION CO2_S3_INTEGRATION;



In [ ]:
LIST @CO2_STAGE;


In [ ]:
//temp
COPY INTO CO2_DATA
FROM @CO2_STAGE/raw_data/co2_data_v2.csv
FILE_FORMAT = (FORMAT_NAME = 'CO2_CSV_FORMAT')
FORCE = TRUE;


In [ ]:
CREATE OR REPLACE FILE FORMAT CO2_CSV_FORMAT
TYPE = CSV
FIELD_DELIMITER = ','
SKIP_HEADER = 1
NULL_IF = ('NULL', 'null', '')
EMPTY_FIELD_AS_NULL = TRUE;


In [ ]:
DESC FILE FORMAT CO2_CSV_FORMAT;


In [ ]:
CREATE OR REPLACE TABLE RAW_DOW30_STAGING (
    YEAR INT,
    MONTH INT,
    DAY INT,
    DECIMAL_DATE FLOAT,
    CO2_EMISSION FLOAT
);


In [ ]:
DESC TABLE RAW_DOW30_STAGING;


In [ ]:
COPY INTO RAW_DOW30_STAGING
FROM @CO2_STAGE/raw_data/co2_data_v2.csv
FILE_FORMAT = (FORMAT_NAME = 'CO2_CSV_FORMAT')
FORCE = TRUE;


In [ ]:
SELECT * FROM RAW_DOW30_STAGING LIMIT 10;


In [ ]:
CREATE OR REPLACE STREAM CO2_STREAM 
ON TABLE RAW_DOW30_STAGING;


In [ ]:
SELECT * FROM CO2_STREAM;


In [ ]:
CREATE OR REPLACE TABLE DOW30_HARMONIZED AS
SELECT 
    YEAR,
    MONTH,
    DAY,
    DECIMAL_DATE,
    CO2_EMISSION
FROM RAW_DOW30_STAGING;


In [ ]:
SELECT * FROM DOW30_HARMONIZED LIMIT 10;


In [ ]:
CREATE OR REPLACE FUNCTION NORMALIZE_CO2_EMISSION(CO2_VALUE FLOAT, REFERENCE_VALUE FLOAT)
RETURNS FLOAT
AS 
$$
    CO2_VALUE / REFERENCE_VALUE
$$;


In [ ]:
SELECT NORMALIZE_CO2_EMISSION(420.15, 410.0) AS NORMALIZED_CO2;


In [ ]:
CREATE OR REPLACE FUNCTION NORMALIZE_CO2_EMISSION(CO2_VALUE FLOAT, REFERENCE_VALUE FLOAT)
RETURNS FLOAT
AS 
$$
    CO2_VALUE / REFERENCE_VALUE
$$;


In [ ]:
SELECT * FROM RAW_DOW30_STAGING LIMIT 10;


In [ ]:
DESC TABLE DOW30_HARMONIZED;


In [ ]:
ALTER TABLE DOW30_HARMONIZED ADD COLUMN NORMALIZED_CO2 FLOAT;


In [ ]:
INSERT INTO DOW30_HARMONIZED (YEAR, MONTH, DAY, CO2_EMISSION, NORMALIZED_CO2)
SELECT 
    YEAR, 
    MONTH, 
    DAY, 
    CO2_EMISSION,
    NORMALIZE_CO2_EMISSION(CO2_EMISSION, 400.0) AS NORMALIZED_CO2  -- Reference value
FROM RAW_DOW30_STAGING;


In [ ]:
SELECT * FROM DOW30_HARMONIZED LIMIT 10;


In [ ]:
SHOW USER FUNCTIONS;


In [ ]:
SELECT NORMALIZE_CO2_EMISSION(333.46, 400);


In [ ]:
UPDATE DOW30_HARMONIZED
SET NORMALIZED_CO2 = NORMALIZE_CO2_EMISSION(CO2_EMISSION, 400.0);


In [ ]:
SELECT * FROM DOW30_HARMONIZED LIMIT 10;


In [ ]:
CREATE OR REPLACE TABLE ANALYTICS_DOW30 AS
SELECT 
    YEAR, 
    MONTH, 
    AVG(CO2_EMISSION) AS AVG_CO2, 
    MAX(CO2_EMISSION) AS MAX_CO2, 
    MIN(CO2_EMISSION) AS MIN_CO2
FROM DOW30_HARMONIZED
GROUP BY YEAR, MONTH;


In [ ]:
SELECT * FROM ANALYTICS_DOW30 LIMIT 10;


In [ ]:
CREATE OR REPLACE PROCEDURE UPDATE_DOW30_SP()
RETURNS STRING
LANGUAGE SQL
AS 
$$
BEGIN
    MERGE INTO DOW30_HARMONIZED AS TARGET
    USING (SELECT * FROM CO2_STREAM) AS SOURCE
    ON TARGET.YEAR = SOURCE.YEAR
    WHEN MATCHED THEN 
        UPDATE SET TARGET.CO2_EMISSION = SOURCE.CO2_EMISSION
    WHEN NOT MATCHED THEN 
        INSERT (YEAR, MONTH, DAY, DECIMAL_DATE, CO2_EMISSION) 
        VALUES (SOURCE.YEAR, SOURCE.MONTH, SOURCE.DAY, SOURCE.DECIMAL_DATE, SOURCE.CO2_EMISSION);
    
    RETURN 'Update Successful!';
END;
$$;


In [ ]:
//stored procedure ...edit here
CREATE OR REPLACE PROCEDURE UPDATE_DOW30_SP()
RETURNS STRING
LANGUAGE SQL
AS 
$$
BEGIN
    MERGE INTO DOW30_HARMONIZED AS TARGET
    USING (SELECT * FROM CO2_STREAM) AS SOURCE
    ON TARGET.YEAR = SOURCE.YEAR
    WHEN MATCHED THEN 
        UPDATE SET TARGET.CO2_EMISSION = SOURCE.CO2_EMISSION
    WHEN NOT MATCHED THEN 
        INSERT (YEAR, MONTH, DAY, DECIMAL_DATE, CO2_EMISSION) 
        VALUES (SOURCE.YEAR, SOURCE.MONTH, SOURCE.DAY, SOURCE.DECIMAL_DATE, SOURCE.CO2_EMISSION);
    
    RETURN 'Update Successful!';
END;
$$;


In [ ]:
CALL UPDATE_DOW30_SP();


In [ ]:
CREATE OR REPLACE TASK UPDATE_DOW30_METRICS_TASK
SCHEDULE = 'USING CRON 0 12 * * * UTC'
AS 
CALL UPDATE_DOW30_SP();


In [ ]:
ALTER TASK UPDATE_DOW30_METRICS_TASK RESUME;


In [ ]:
SELECT COUNT(*) FROM DOW30_HARMONIZED;
